In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
split = 2

In [3]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1428482/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1428482/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.6196185265268598,
 'decoded_ood_sinkhorn_div_1': 117.42914581298828,
 'decoded_ood_sinkhorn_div_10': 83.74657767159599,
 'decoded_ood_sinkhorn_div_100': 37.026117597307476,
 'decoded_ood_e_distance': 73.17683437227947,
 'decoded_ood_mmd': 0.521114706993103}

In [14]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7530552869041761,
 'decoded_test_e_distance': 48.60821243706078,
 'decoded_test_mmd_distance': 0.5026096676786741}

In [15]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.6196185265268598,
 'decoded_ood_sinkhorn_div_1': 117.42914581298828,
 'decoded_ood_sinkhorn_div_10': 83.74657767159599,
 'decoded_ood_sinkhorn_div_100': 37.026117597307476,
 'decoded_ood_e_distance': 73.17683437227947,
 'decoded_ood_mmd': 0.521114706993103}

In [16]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -2.7975080013275146,
 'encoded_ood_sinkhorn_div_1': 40.55727965491159,
 'encoded_ood_sinkhorn_div_10': 36.5319607598441,
 'encoded_ood_sinkhorn_div_100': 35.25404030936105,
 'encoded_ood_e_distance': 70.20893376716798,
 'encoded_ood_mmd': 0.526747852563858}

In [17]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.34591197967529297,
 'deg_ood_sinkhorn_div_1': 31.23884037562779,
 'deg_ood_sinkhorn_div_10': 22.39100306374686,
 'deg_ood_sinkhorn_div_100': 21.720914840698242,
 'deg_ood_e_distance': 43.30413815593588,
 'deg_ood_mmd': 0.5662017209189278}

In [18]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5647958442568779,
 'deg_test_sinkhorn_div_1': 23.240078111489613,
 'deg_test_sinkhorn_div_10': 14.815709034601847,
 'deg_test_sinkhorn_div_100': 13.781193673610687,
 'deg_test_e_distance': 27.41106275169496,
 'deg_test_mmd': 0.5228426506121954}

In [19]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7530552869041761,
 'decoded_test_e_distance': 48.60821243706078,
 'decoded_test_mmd_distance': 0.5026096676786741}

In [20]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.6196185265268598,
 'decoded_ood_sinkhorn_div_1': 117.42914581298828,
 'decoded_ood_sinkhorn_div_10': 83.74657767159599,
 'decoded_ood_sinkhorn_div_100': 37.026117597307476,
 'decoded_ood_e_distance': 73.17683437227947,
 'decoded_ood_mmd': 0.521114706993103}

In [21]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.34591197967529297,
 'deg_ood_sinkhorn_div_1': 31.23884037562779,
 'deg_ood_sinkhorn_div_10': 22.39100306374686,
 'deg_ood_sinkhorn_div_100': 21.720914840698242,
 'deg_ood_e_distance': 43.30413815593588,
 'deg_ood_mmd': 0.5662017209189278}

In [22]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7530552869041761,
 'decoded_test_e_distance': 48.60821243706078,
 'decoded_test_mmd_distance': 0.5026096676786741}

In [23]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.34101922409728863,
 'encoded_test_sinkhorn_div_1': 28.811219731966656,
 'encoded_test_sinkhorn_div_10': 24.24452479680379,
 'encoded_test_sinkhorn_div_100': 22.821958263715107,
 'encoded_test_e_distance': 45.31849273684808,
 'encoded_test_mmd': 0.4645499251782894}

In [24]:
ood_metrics_encoded

{'Dacinostat+Dasatinib': {'r_squared': -5.884942531585693,
  'sinkhorn_div_1': 33.454647064208984,
  'sinkhorn_div_10': 29.70547866821289,
  'sinkhorn_div_100': 28.864126205444336,
  'e_distance': 57.559975048071706,
  'mmd': 0.5504406},
 'Dacinostat+PCI-34051': {'r_squared': -5.991818428039551,
  'sinkhorn_div_1': 35.791961669921875,
  'sinkhorn_div_10': 32.12656784057617,
  'sinkhorn_div_100': 31.28960418701172,
  'e_distance': 62.402391225021894,
  'mmd': 0.5601594},
 'Givinostat+Cediranib': {'r_squared': 0.7920493483543396,
  'sinkhorn_div_1': 7.482573509216309,
  'sinkhorn_div_10': 4.1853532791137695,
  'sinkhorn_div_100': 3.058253288269043,
  'e_distance': 5.846879899766877,
  'mmd': 0.32691115},
 'Givinostat+Curcumin': {'r_squared': 0.9357357621192932,
  'sinkhorn_div_1': 5.6534037590026855,
  'sinkhorn_div_10': 2.30694580078125,
  'sinkhorn_div_100': 1.078455924987793,
  'e_distance': 1.8456988253423992,
  'mmd': 0.3018954},
 'Panobinostat+Alvespimycin': {'r_squared': -2.253347

In [25]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/chemcpa"

In [26]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
